In [1]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn import linear_model
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn import svm
from math import floor
import seaborn as sns
from math import ceil
import pandas as pd
import numpy as np

pokedex = pd.read_csv('pokemon.csv')
pokedex

,abilities,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,...,percentage_male,pokedex_number,sp_attack,sp_defense,speed,type1,type2,weight_kg,generation,is_legendary
0,"['Overgrow', 'Chlorophyll']",1.00,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,1,65,65,45,grass,poison,6.9,1,0
1,"['Overgrow', 'Chlorophyll']",1.00,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,2,80,80,60,grass,poison,13.0,1,0
2,"['Overgrow', 'Chlorophyll']",1.00,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,3,122,120,80,grass,poison,100.0,1,0
3,"['Blaze', 'Solar Power']",0.50,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,4,60,50,65,fire,NaN,8.5,1,0
4,"['Blaze', 'Solar Power']",0.50,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,5,80,65,80,fire,NaN,19.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,['Beast Boost'],0.25,1.0,0.5,2.0,0.5,1.0,2.0,0.5,1.0,...,NaN,797,107,101,61,steel,flying,999.9,7,1
797,['Beast Boost'],1.00,1.0,0.5,0.5,0.5,2.0,4.0,1.0,1.0,...,NaN,798,59,31,109,grass,steel,0.1,7,1
798,['Beast Boost'],2.00,0.5,2.0,0.5,4.0,2.0,0.5,1.0,0.5,...,NaN,799,97,53,43,dark,dragon,888.0,7,1
799,['Prism Armor'],2.00,2.0,1.0,1.0,1.0,0.5,1.0,1.0,2.0,...,NaN,800,127,89,79,psychic,NaN,230.0,7,1


In [2]:
pokedex.name[6]

'Squirtle'

In [3]:
lista = list(zip(pokedex.against_bug, pokedex.against_dark, pokedex.against_dragon, pokedex.against_electric, pokedex.against_fairy, 
                 pokedex.against_fight, pokedex.against_fire, pokedex.against_flying, pokedex.against_ghost, pokedex.against_grass, 
                 pokedex.against_ground, pokedex.against_ice, pokedex.against_normal, pokedex.against_poison, pokedex.against_psychic, 
                 pokedex.against_rock, pokedex.against_steel, pokedex.against_water))

VeD = [list(item) for item in lista]
print(VeD[34])

[0.5, 0.5, 0.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0]


In [4]:
tipo1 = pokedex.type1
encoder = LabelEncoder()
tipo1 = encoder.fit_transform(tipo1)

tipo2 = pokedex.type2
encoder = LabelEncoder()
tipo2 = encoder.fit_transform(tipo2)

tipagem = set(pokedex.type1)
tipagem = sorted(list(tipagem))
tipagem.append('pure')
tipagem

['bug',
 'dark',
 'dragon',
 'electric',
 'fairy',
 'fighting',
 'fire',
 'flying',
 'ghost',
 'grass',
 'ground',
 'ice',
 'normal',
 'poison',
 'psychic',
 'rock',
 'steel',
 'water',
 'pure']

In [5]:
tipoum = []
tipodois = []

for i in range(0,len(tipo1)):
    tipoum.append(int(tipo1[i]))
    i += 1
    
for i in range(0,len(tipo2)):
    tipodois.append(int(tipo2[i]))
    i += 1

junta_tipos = list(zip(tipoum, tipodois))

tipos = [list(item) for item in junta_tipos]

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# separa os dados que vão para treino e teste
# sendo 70% desses dados para treino e 30% para teste
entrada_treino, entrada_teste, saida_treino, saida_teste = train_test_split(
    VeD, tipos, test_size=0.2, random_state=0)

# mostra que a quantidade de dados foi certa para cada parametro
print(len(entrada_treino))
print(len(entrada_teste))

640
161


REGRESSÃO LINEAR

In [7]:
# comandos para importar e realizar a regressão linear
regressao = linear_model.LinearRegression()
regressao.fit(entrada_treino, saida_treino)

LinearRegression()

In [8]:
a = regressao.coef_  # "a" é o coeficiente angular para a equação da reta
b = regressao.intercept_  # "b" é o coeficiente linear para a equação da reta

print("Coef Angular:", a, "Coef Linear:", b)

Coef Angular: [[ 0.01844542  3.15951353  0.57860104  4.25190021 -5.15360875  2.74259993
  -1.84287543  0.60103138  0.90700537  1.41108563 -0.90474744  1.09490971
   2.08291826  0.03148996  0.00719677 -4.27424211  1.6577205  -1.02708825]
 [ 0.7425521   0.13539029  5.28731811  0.37249909 -4.0799977   1.06623875
   1.1043566  -1.38961139  1.27626888 -0.38215151 -1.49571505 -3.39134764
   4.21156293  1.1601044   1.6009082  -3.51092621  0.71111226  0.61786378]] Coef Linear: [ 5.26379361 11.89755333]


In [9]:
# mostra a precisão da regressão linear
print(f"Precisão: {regressao.score(entrada_treino, saida_treino) * 100:.2f}%")

Precisão: 52.33%


In [10]:
predicao = regressao.predict(entrada_teste)

print(f'MSE = {mean_squared_error(y_true=saida_teste, y_pred=predicao)}') # raiz disso mostra uma média de desvio dos resultados (+-x)
print(f'R2 = {r2_score(y_true=saida_teste, y_pred=predicao)}') # raiz disso mostra a porcentagem de acertos
print(f'MAE = {mean_absolute_error(y_true=saida_teste, y_pred=predicao)}')

MSE = 16.136414847196644
R2 = 0.49556902669995945
MAE = 3.058733980531442


In [11]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = regressao.predict([VeD[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = regressao.predict([VeD[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = regressao.predict([VeD[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = regressao.predict([VeD[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = regressao.predict([VeD[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = regressao.predict([VeD[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = regressao.predict([VeD[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = regressao.predict([VeD[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = regressao.predict([VeD[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = regressao.predict([VeD[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = regressao.predict([VeD[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = regressao.predict([VeD[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = regressao.predict([VeD[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = regressao.predict([VeD[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = regressao.predict([VeD[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = regressao.predict([VeD[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = regressao.predict([VeD[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = regressao.predict([VeD[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = regressao.predict([[1.0, 1.0, 1.0, 0.5, 0.5, 0.5, 2.0, 2.0, 1.0, 0.25, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.5]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = regressao.predict([[0.5, 0.5, 2.0, 0.5, 4.0, 1.0, 0.5, 2.0, 1.0, 0.5, 1.0, 2.0, 1.0, 1.0, 2.0, 0.5, 1.0, 0.5]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 3.46509412 12.54361214]]
Este pokémon provavelmente é do tipo: electric/normal

Entrando com dados do tipo dark
[[ 6.17262973 11.35603459]]
Este pokémon provavelmente é do tipo: fire/ice

Entrando com dados do tipo dragon
[[ 5.72904026 12.89366803]]
Este pokémon provavelmente é do tipo: fighting/normal

Entrando com dados do tipo electric
[[ 6.44557585 14.5912642 ]]
Este pokémon provavelmente é do tipo: fire/psychic

Entrando com dados do tipo fairy
[[ 8.75597933 11.54578721]]
Este pokémon provavelmente é do tipo: ghost/ice

Entrando com dados do tipo fighting
[[ 6.60841033 13.38177025]]
Este pokémon provavelmente é do tipo: fire/poison

Entrando com dados do tipo fire
[[ 5.80673299 14.1929397 ]]
Este pokémon provavelmente é do tipo: fighting/psychic

Entrando com dados do tipo flying
[[10.4968991  10.18659985]]
Este pokémon provavelmente é do tipo: ground/ground

Entrando com dados do tipo ghost
[[ 9.82168236 11.11650847]]
Este pokémon provavelmente é

KNN

In [12]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=2)
knn.fit(entrada_treino, saida_treino)


KNeighborsRegressor(n_neighbors=2)

In [13]:
# mostra a precisão do KNN
print(f"Precisão: {knn.score(entrada_treino, saida_treino) * 100:.2f}%")

Precisão: 81.55%


In [14]:
predicao = knn.predict(entrada_teste)

print(f'MSE = {mean_squared_error(y_true=saida_teste, y_pred=predicao)}')
print(f'R2 = {r2_score(y_true=saida_teste, y_pred=predicao)}')
print(f'MAE = {mean_absolute_error(y_true=saida_teste, y_pred=predicao)}')

MSE = 13.560559006211179
R2 = 0.5788037118687401
MAE = 1.658385093167702


In [15]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = knn.predict([VeD[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = knn.predict([VeD[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = knn.predict([VeD[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = knn.predict([VeD[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = knn.predict([VeD[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = knn.predict([VeD[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = knn.predict([VeD[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = knn.predict([VeD[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = knn.predict([VeD[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = knn.predict([VeD[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = knn.predict([VeD[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = knn.predict([VeD[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = knn.predict([VeD[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = knn.predict([VeD[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = knn.predict([VeD[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = knn.predict([VeD[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = knn.predict([VeD[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = knn.predict([VeD[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = knn.predict([[1.0, 1.0, 1.0, 0.5, 0.5, 0.5, 2.0, 2.0, 1.0, 0.25, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.5]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = knn.predict([[0.5, 0.5, 2.0, 0.5, 4.0, 1.0, 0.5, 2.0, 1.0, 0.5, 1.0, 2.0, 1.0, 1.0, 2.0, 0.5, 1.0, 0.5]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 0. 18.]]
Este pokémon provavelmente é do tipo: bug/pure

Entrando com dados do tipo dark
[[ 1. 18.]]
Este pokémon provavelmente é do tipo: dark/pure

Entrando com dados do tipo dragon
[[ 2. 18.]]
Este pokémon provavelmente é do tipo: dragon/pure

Entrando com dados do tipo electric
[[ 3. 18.]]
Este pokémon provavelmente é do tipo: electric/pure

Entrando com dados do tipo fairy
[[ 4. 18.]]
Este pokémon provavelmente é do tipo: fairy/pure

Entrando com dados do tipo fighting
[[ 5. 18.]]
Este pokémon provavelmente é do tipo: fighting/pure

Entrando com dados do tipo fire
[[ 6. 18.]]
Este pokémon provavelmente é do tipo: fire/pure

Entrando com dados do tipo flying
[[ 9.5 12.5]]
Este pokémon provavelmente é do tipo: grass/normal

Entrando com dados do tipo ghost
[[ 8. 18.]]
Este pokémon provavelmente é do tipo: ghost/pure

Entrando com dados do tipo grass
[[ 9. 18.]]
Este pokémon provavelmente é do tipo: grass/pure

Entrando com dados do tipo ground
[[10.

RANDOM FOREST

In [16]:
rf = RandomForestRegressor()
#rf = RandomForestClassifier(n_estimators=1500, criterion='entropy', min_samples_split=10, max_depth=24, random_state=2)
rf.fit(entrada_treino, saida_treino)

RandomForestRegressor()

In [17]:
print(f'{rf.score(entrada_teste, saida_teste) * 100:.2f}%')

81.38%


In [18]:
predicao = rf.predict(entrada_teste)

print(f'MSE = {mean_squared_error(y_true=saida_teste, y_pred=predicao)}')
print(f'R2 = {r2_score(y_true=saida_teste, y_pred=predicao)}')
print(f'MAE = {mean_absolute_error(y_true=saida_teste, y_pred=predicao)}')

MSE = 5.938585748611749
R2 = 0.8137626183056812
MAE = 1.1182077995654476


In [19]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = rf.predict([VeD[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = rf.predict([VeD[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = rf.predict([VeD[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = rf.predict([VeD[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = rf.predict([VeD[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = rf.predict([VeD[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = rf.predict([VeD[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = rf.predict([VeD[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = rf.predict([VeD[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = rf.predict([VeD[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = rf.predict([VeD[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = rf.predict([VeD[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = rf.predict([VeD[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = rf.predict([VeD[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = rf.predict([VeD[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = rf.predict([VeD[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = rf.predict([VeD[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = rf.predict([VeD[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = rf.predict([[1.0, 1.0, 1.0, 0.5, 0.5, 0.5, 2.0, 2.0, 1.0, 0.25, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.5]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = rf.predict([[0.5, 0.5, 2.0, 0.5, 4.0, 1.0, 0.5, 2.0, 1.0, 0.5, 1.0, 2.0, 1.0, 1.0, 2.0, 0.5, 1.0, 0.5]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 0. 18.]]
Este pokémon provavelmente é do tipo: bug/pure

Entrando com dados do tipo dark
[[ 1. 18.]]
Este pokémon provavelmente é do tipo: dark/pure

Entrando com dados do tipo dragon
[[ 2. 18.]]
Este pokémon provavelmente é do tipo: dragon/pure

Entrando com dados do tipo electric
[[ 3.         17.25136798]]
Este pokémon provavelmente é do tipo: electric/water

Entrando com dados do tipo fairy
[[ 4. 18.]]
Este pokémon provavelmente é do tipo: fairy/pure

Entrando com dados do tipo fighting
[[ 5. 18.]]
Este pokémon provavelmente é do tipo: fighting/pure

Entrando com dados do tipo fire
[[ 6.        17.1510314]]
Este pokémon provavelmente é do tipo: fire/water

Entrando com dados do tipo flying
[[ 8.87 14.15]]
Este pokémon provavelmente é do tipo: ghost/psychic

Entrando com dados do tipo ghost
[[ 8. 18.]]
Este pokémon provavelmente é do tipo: ghost/pure

Entrando com dados do tipo grass
[[ 9. 18.]]
Este pokémon provavelmente é do tipo: grass/pure

Entr

DECISION TREE

In [20]:
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor()
dtr.fit(entrada_treino, saida_treino)

DecisionTreeRegressor()

In [21]:
print(f'{dtr.score(entrada_teste, saida_teste)* 100:.2f}%')

80.18%


In [22]:
predicao = dtr.predict(entrada_teste)

print(f'MSE = {mean_squared_error(y_true=saida_teste, y_pred=predicao)}')
print(f'R2 = {r2_score(y_true=saida_teste, y_pred=predicao)}')
print(f'MAE = {mean_absolute_error(y_true=saida_teste, y_pred=predicao)}')

MSE = 6.311759554209166
R2 = 0.8018010094845474
MAE = 0.9412188309705971


In [23]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = dtr.predict([VeD[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = dtr.predict([VeD[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = dtr.predict([VeD[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = dtr.predict([VeD[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = dtr.predict([VeD[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = dtr.predict([VeD[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = dtr.predict([VeD[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = dtr.predict([VeD[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = dtr.predict([VeD[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = dtr.predict([VeD[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = dtr.predict([VeD[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = dtr.predict([VeD[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = dtr.predict([VeD[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = dtr.predict([VeD[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = dtr.predict([VeD[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = dtr.predict([VeD[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = dtr.predict([VeD[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = dtr.predict([VeD[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = dtr.predict([[1.0, 1.0, 1.0, 0.5, 0.5, 0.5, 2.0, 2.0, 1.0, 0.25, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 0.5]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = dtr.predict([[0.5, 0.5, 2.0, 0.5, 4.0, 1.0, 0.5, 2.0, 1.0, 0.5, 1.0, 2.0, 1.0, 1.0, 2.0, 0.5, 1.0, 0.5]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 0. 18.]]
Este pokémon provavelmente é do tipo: bug/pure

Entrando com dados do tipo dark
[[ 1. 18.]]
Este pokémon provavelmente é do tipo: dark/pure

Entrando com dados do tipo dragon
[[ 2. 18.]]
Este pokémon provavelmente é do tipo: dragon/pure

Entrando com dados do tipo electric
[[ 3.         17.21052632]]
Este pokémon provavelmente é do tipo: electric/water

Entrando com dados do tipo fairy
[[ 4. 18.]]
Este pokémon provavelmente é do tipo: fairy/pure

Entrando com dados do tipo fighting
[[ 5. 18.]]
Este pokémon provavelmente é do tipo: fighting/pure

Entrando com dados do tipo fire
[[ 6.         17.17391304]]
Este pokémon provavelmente é do tipo: fire/water

Entrando com dados do tipo flying
[[ 7. 18.]]
Este pokémon provavelmente é do tipo: flying/pure

Entrando com dados do tipo ghost
[[ 8. 18.]]
Este pokémon provavelmente é do tipo: ghost/pure

Entrando com dados do tipo grass
[[ 9. 18.]]
Este pokémon provavelmente é do tipo: grass/pure

Entrando

------------- PCA -------------

In [24]:
pca = PCA(n_components=2)
duas_dimensoes = pca.fit_transform(VeD)
duas_dimensoes

array([[-1.45577192, -0.35427967],
       [-1.45577192, -0.35427967],
       [-1.45577192, -0.35427967],
       ...,
       [-0.26567667, -2.17444328],
       [-0.10206668, -0.07391654],
       [ 0.96713856,  0.39104124]])

In [25]:
# separa os dados que vão para treino e teste
# sendo 70% desses dados para treino e 30% para teste
entrada_treinoPCA, entrada_testePCA, saida_treinoPCA, saida_testePCA = train_test_split(
    duas_dimensoes, tipos, test_size=0.2, random_state=0)

# mostra que a quantidade de dados foi certa para cada parametro
print(len(entrada_treinoPCA))
print(len(entrada_testePCA))

640
161


REGRESSÃO LINEAR COM PCA

In [26]:
# comandos para importar e realizar a regressão linear
regressaoPCA = linear_model.LinearRegression()
regressaoPCA.fit(entrada_treinoPCA, saida_treinoPCA)

LinearRegression()

In [27]:
a = regressaoPCA.coef_  # "a" é o coeficiente angular para a equação da reta
b = regressaoPCA.intercept_  # "b" é o coeficiente linear para a equação da reta
print("Coef Angular:", a, "Coef Linear:", b)

Coef Angular: [[ 1.80533014  0.5608609 ]
 [ 1.12332095 -0.01783421]] Coef Linear: [ 9.59778875 13.17186911]


In [28]:
# mostra a precisão da regressão linear
print(f"Precisão: {regressaoPCA.score(entrada_treinoPCA, saida_treinoPCA) * 100:.2f}%")

Precisão: 12.16%


In [29]:
predicao = regressaoPCA.predict(entrada_testePCA)

print(f'MSE = {mean_squared_error(y_true=saida_testePCA, y_pred=predicao)}')  # raiz disso mostra uma média de desvio dos resultados (+-x)
print(f'R2 = {r2_score(y_true=saida_testePCA, y_pred=predicao)}')  # raiz disso mostra a porcentagem de acertos
print(f'MAE = {mean_absolute_error(y_true=saida_testePCA, y_pred=predicao)}')

MSE = 28.964454974641907
R2 = 0.08695571545974728
MAE = 4.721433156327266


In [30]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = regressaoPCA.predict([duas_dimensoes[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = regressaoPCA.predict([duas_dimensoes[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = regressaoPCA.predict([duas_dimensoes[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = regressaoPCA.predict([duas_dimensoes[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = regressaoPCA.predict([duas_dimensoes[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = regressaoPCA.predict([duas_dimensoes[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = regressaoPCA.predict([duas_dimensoes[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = regressaoPCA.predict([duas_dimensoes[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = regressaoPCA.predict([duas_dimensoes[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = regressaoPCA.predict([duas_dimensoes[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = regressaoPCA.predict([duas_dimensoes[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = regressaoPCA.predict([duas_dimensoes[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = regressaoPCA.predict([duas_dimensoes[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = regressaoPCA.predict([duas_dimensoes[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = regressaoPCA.predict([duas_dimensoes[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = regressaoPCA.predict([duas_dimensoes[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = regressaoPCA.predict([duas_dimensoes[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = regressaoPCA.predict([duas_dimensoes[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = regressaoPCA.predict([duas_dimensoes[0]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = regressaoPCA.predict([duas_dimensoes[783]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 7.37073518 11.63794497]]
Este pokémon provavelmente é do tipo: flying/ice

Entrando com dados do tipo dark
[[ 9.59923638 13.60143893]]
Este pokémon provavelmente é do tipo: grass/poison

Entrando com dados do tipo dragon
[[ 8.39103682 12.69722511]]
Este pokémon provavelmente é do tipo: ghost/normal

Entrando com dados do tipo electric
[[10.94265862 14.05775596]]
Este pokémon provavelmente é do tipo: ground/psychic

Entrando com dados do tipo fairy
[[ 9.26849613 12.9380703 ]]
Este pokémon provavelmente é do tipo: grass/normal

Entrando com dados do tipo fighting
[[ 9.13393721 13.00881448]]
Este pokémon provavelmente é do tipo: grass/poison

Entrando com dados do tipo fire
[[11.65682185 14.0348088 ]]
Este pokémon provavelmente é do tipo: ice/psychic

Entrando com dados do tipo flying
[[ 7.9717528  11.78174299]]
Este pokémon provavelmente é do tipo: flying/ice

Entrando com dados do tipo ghost
[[10.11015341 13.20278672]]
Este pokémon provavelmente é do ti

KNN COM PCA

In [31]:
knnPCA = KNeighborsRegressor(n_neighbors=3)
knnPCA.fit(entrada_treinoPCA, saida_treinoPCA)

KNeighborsRegressor(n_neighbors=3)

In [32]:
# mostra a precisão
print(f"Precisão: {knnPCA.score(entrada_treinoPCA, saida_treinoPCA) * 100:.2f}%")
predicao = knnPCA.predict(entrada_testePCA)

Precisão: 82.26%


In [33]:
print(f'MSE = {mean_squared_error(y_true=saida_testePCA, y_pred=predicao)}')  # raiz disso mostra uma média de desvio dos resultados (+-x)
print(f'R2 = {r2_score(y_true=saida_testePCA, y_pred=predicao)}')  # raiz disso mostra a porcentagem de acertos
print(f'MAE = {mean_absolute_error(y_true=saida_testePCA, y_pred=predicao)}')

MSE = 12.489648033126294
R2 = 0.605338652622841
MAE = 1.7028985507246375


In [34]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = knnPCA.predict([duas_dimensoes[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = knnPCA.predict([duas_dimensoes[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = knnPCA.predict([duas_dimensoes[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = knnPCA.predict([duas_dimensoes[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = knnPCA.predict([duas_dimensoes[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = knnPCA.predict([duas_dimensoes[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = knnPCA.predict([duas_dimensoes[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = knnPCA.predict([duas_dimensoes[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = knnPCA.predict([duas_dimensoes[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = knnPCA.predict([duas_dimensoes[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = knnPCA.predict([duas_dimensoes[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = knnPCA.predict([duas_dimensoes[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = knnPCA.predict([duas_dimensoes[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = knnPCA.predict([duas_dimensoes[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = knnPCA.predict([duas_dimensoes[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = knnPCA.predict([duas_dimensoes[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = knnPCA.predict([duas_dimensoes[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = knnPCA.predict([duas_dimensoes[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = knnPCA.predict([duas_dimensoes[0]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = knnPCA.predict([duas_dimensoes[783]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 0. 18.]]
Este pokémon provavelmente é do tipo: bug/pure

Entrando com dados do tipo dark
[[ 1. 18.]]
Este pokémon provavelmente é do tipo: dark/pure

Entrando com dados do tipo dragon
[[ 2. 18.]]
Este pokémon provavelmente é do tipo: dragon/pure

Entrando com dados do tipo electric
[[ 3. 18.]]
Este pokémon provavelmente é do tipo: electric/pure

Entrando com dados do tipo fairy
[[ 4. 18.]]
Este pokémon provavelmente é do tipo: fairy/pure

Entrando com dados do tipo fighting
[[ 5. 18.]]
Este pokémon provavelmente é do tipo: fighting/pure

Entrando com dados do tipo fire
[[ 6. 18.]]
Este pokémon provavelmente é do tipo: fire/pure

Entrando com dados do tipo flying
[[ 6.66666667 10.33333333]]
Este pokémon provavelmente é do tipo: fire/ground

Entrando com dados do tipo ghost
[[ 8. 18.]]
Este pokémon provavelmente é do tipo: ghost/pure

Entrando com dados do tipo grass
[[ 9. 18.]]
Este pokémon provavelmente é do tipo: grass/pure

Entrando com dados do tipo

RANDOM FOREST COM PCA

In [35]:
rfPCA = RandomForestRegressor()
rfPCA.fit(entrada_treinoPCA, saida_treinoPCA)

RandomForestRegressor()

In [36]:
# mostra a precisão
print(f"Precisão: {rfPCA.score(entrada_treinoPCA, saida_treinoPCA) * 100:.2f}%")

Precisão: 90.66%


In [37]:
predicao = rfPCA.predict(entrada_testePCA)
print(f'MSE = {mean_squared_error(y_true=saida_testePCA, y_pred=predicao)}')  # raiz disso mostra uma média de desvio dos resultados (+-x)
print(f'R2 = {r2_score(y_true=saida_testePCA, y_pred=predicao)}')  # raiz disso mostra a porcentagem de acertos
print(f'MAE = {mean_absolute_error(y_true=saida_testePCA, y_pred=predicao)}')

MSE = 7.044146847021288
R2 = 0.7791357267094974
MAE = 1.2313525387367945


In [38]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = rfPCA.predict([duas_dimensoes[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = rfPCA.predict([duas_dimensoes[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = rfPCA.predict([duas_dimensoes[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = rfPCA.predict([duas_dimensoes[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = rfPCA.predict([duas_dimensoes[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = rfPCA.predict([duas_dimensoes[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = rfPCA.predict([duas_dimensoes[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = rfPCA.predict([duas_dimensoes[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = rfPCA.predict([duas_dimensoes[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = rfPCA.predict([duas_dimensoes[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = rfPCA.predict([duas_dimensoes[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = rfPCA.predict([duas_dimensoes[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = rfPCA.predict([duas_dimensoes[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = rfPCA.predict([duas_dimensoes[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = rfPCA.predict([duas_dimensoes[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = rfPCA.predict([duas_dimensoes[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = rfPCA.predict([duas_dimensoes[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = rfPCA.predict([duas_dimensoes[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = rfPCA.predict([duas_dimensoes[0]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = rfPCA.predict([duas_dimensoes[783]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 0. 18.]]
Este pokémon provavelmente é do tipo: bug/pure

Entrando com dados do tipo dark
[[ 1. 18.]]
Este pokémon provavelmente é do tipo: dark/pure

Entrando com dados do tipo dragon
[[ 2. 18.]]
Este pokémon provavelmente é do tipo: dragon/pure

Entrando com dados do tipo electric
[[ 3.        17.2548716]]
Este pokémon provavelmente é do tipo: electric/water

Entrando com dados do tipo fairy
[[ 4. 18.]]
Este pokémon provavelmente é do tipo: fairy/pure

Entrando com dados do tipo fighting
[[ 5. 18.]]
Este pokémon provavelmente é do tipo: fighting/pure

Entrando com dados do tipo fire
[[ 6.         17.15110396]]
Este pokémon provavelmente é do tipo: fire/water

Entrando com dados do tipo flying
[[ 5.94026659 15.69973341]]
Este pokémon provavelmente é do tipo: fighting/rock

Entrando com dados do tipo ghost
[[ 8. 18.]]
Este pokémon provavelmente é do tipo: ghost/pure

Entrando com dados do tipo grass
[[ 9. 18.]]
Este pokémon provavelmente é do tipo: gras

DECISION TREE COM PCA

In [39]:
dtrPCA = DecisionTreeRegressor()
dtrPCA.fit(entrada_treinoPCA, saida_treinoPCA)

DecisionTreeRegressor()

In [40]:
# mostra a precisão
print(f"Precisão: {dtrPCA.score(entrada_treinoPCA, saida_treinoPCA) * 100:.2f}%")

Precisão: 92.06%


In [41]:
predicao = dtrPCA.predict(entrada_testePCA)
print(f'MSE = {mean_squared_error(y_true=saida_testePCA, y_pred=predicao)}')  # raiz disso mostra uma média de desvio dos resultados (+-x)
print(f'R2 = {r2_score(y_true=saida_testePCA, y_pred=predicao)}')  # raiz disso mostra a porcentagem de acertos
print(f'MAE = {mean_absolute_error(y_true=saida_testePCA, y_pred=predicao)}')

MSE = 7.325517749516274
R2 = 0.7702058410285506
MAE = 1.0298834272438888


In [42]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = dtrPCA.predict([duas_dimensoes[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = dtrPCA.predict([duas_dimensoes[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = dtrPCA.predict([duas_dimensoes[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = dtrPCA.predict([duas_dimensoes[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = dtrPCA.predict([duas_dimensoes[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = dtrPCA.predict([duas_dimensoes[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = dtrPCA.predict([duas_dimensoes[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = dtrPCA.predict([duas_dimensoes[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = dtrPCA.predict([duas_dimensoes[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = dtrPCA.predict([duas_dimensoes[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = dtrPCA.predict([duas_dimensoes[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = dtrPCA.predict([duas_dimensoes[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = dtrPCA.predict([duas_dimensoes[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = dtrPCA.predict([duas_dimensoes[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = dtrPCA.predict([duas_dimensoes[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = dtrPCA.predict([duas_dimensoes[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = dtrPCA.predict([duas_dimensoes[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = dtrPCA.predict([duas_dimensoes[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = dtrPCA.predict([duas_dimensoes[0]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = dtrPCA.predict([duas_dimensoes[783]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 0. 18.]]
Este pokémon provavelmente é do tipo: bug/pure

Entrando com dados do tipo dark
[[ 1. 18.]]
Este pokémon provavelmente é do tipo: dark/pure

Entrando com dados do tipo dragon
[[ 2. 18.]]
Este pokémon provavelmente é do tipo: dragon/pure

Entrando com dados do tipo electric
[[ 3.         17.21052632]]
Este pokémon provavelmente é do tipo: electric/water

Entrando com dados do tipo fairy
[[ 4. 18.]]
Este pokémon provavelmente é do tipo: fairy/pure

Entrando com dados do tipo fighting
[[ 5. 18.]]
Este pokémon provavelmente é do tipo: fighting/pure

Entrando com dados do tipo fire
[[ 6.         17.17391304]]
Este pokémon provavelmente é do tipo: fire/water

Entrando com dados do tipo flying
[[ 7. 18.]]
Este pokémon provavelmente é do tipo: flying/pure

Entrando com dados do tipo ghost
[[ 8. 18.]]
Este pokémon provavelmente é do tipo: ghost/pure

Entrando com dados do tipo grass
[[ 9. 18.]]
Este pokémon provavelmente é do tipo: grass/pure

Entrando

------------- FEATURE SCALING -------------

In [43]:
FS = StandardScaler()
padronizado = FS.fit_transform(VeD)
padronizado

array([[ 0.00627488, -0.13044138,  0.08845721, ..., -0.35927587,
         0.0330966 , -0.92111511],
       [ 0.00627488, -0.13044138,  0.08845721, ..., -0.35927587,
         0.0330966 , -0.92111511],
       [ 0.00627488, -0.13044138,  0.08845721, ..., -0.35927587,
         0.0330966 , -0.92111511],
       ...,
       [ 1.68166846, -1.27233802,  2.92262606, ..., -0.35927587,
         0.0330966 , -0.92111511],
       [ 1.68166846,  2.1533519 ,  0.08845721, ..., -0.35927587,
         0.0330966 , -0.09628202],
       [-1.2502703 , -1.27233802, -2.74571165, ..., -1.07693167,
         0.0330966 , -0.09628202]])

In [44]:
# separa os dados que vão para treino e teste
# sendo 70% desses dados para treino e 30% para teste
entrada_treinoFS, entrada_testeFS, saida_treinoFS, saida_testeFS = train_test_split(
    padronizado, tipos, test_size=0.2, random_state=0)

# mostra que a quantidade de dados foi certa para cada parametro
print(len(entrada_treinoFS))
print(len(entrada_testeFS))

640
161


In [45]:
# comandos para importar e realizar a regressão linear
regressaoFS = linear_model.LinearRegression()
regressaoFS.fit(entrada_treinoFS, saida_treinoFS)

LinearRegression()

In [46]:
a = regressaoFS.coef_  # "a" é o coeficiente angular para a equação da reta
b = regressaoFS.intercept_  # "b" é o coeficiente linear para a equação da reta
print("Coef Angular:", a, "Coef Linear:", b)

Coef Angular: [[ 0.0110096   1.38344987  0.20415193  2.78309255 -2.68936323  1.96590374
  -1.2742021   0.3630893   0.50602544  1.11250464 -0.66802644  0.80464585
   0.55393117  0.017289    0.00356149 -2.97791932  0.82853593 -0.62260369]
 [ 0.44321054  0.05928307  1.86556214  0.24382026 -2.1291092   0.76428309
   0.76357494 -0.83947868  0.71204047 -0.30128953 -1.1043714  -2.49229116
   1.12002283  0.63693468  0.7922475  -2.44610733  0.35541701  0.37453867]] Coef Linear: [ 9.55342565 13.23629795]


In [47]:
# mostra a precisão da regressão linear
print(f"Precisão: {regressaoFS.score(entrada_treinoFS, saida_treinoFS) * 100:.2f}%")

Precisão: 52.33%


In [48]:
predicao = regressaoFS.predict(entrada_testeFS)
print(f'MSE = {mean_squared_error(y_true=saida_testeFS, y_pred=predicao)}')  # raiz disso mostra uma média de desvio dos resultados (+-x)
print(f'R2 = {r2_score(y_true=saida_testeFS, y_pred=predicao)}')  # raiz disso mostra a porcentagem de acertos
print(f'MAE = {mean_absolute_error(y_true=saida_testeFS, y_pred=predicao)}')

MSE = 16.136414847196647
R2 = 0.4955690266999593
MAE = 3.0587339805314424


In [49]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = regressaoFS.predict([padronizado[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = regressaoFS.predict([padronizado[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = regressaoFS.predict([padronizado[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = regressaoFS.predict([padronizado[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = regressaoFS.predict([padronizado[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = regressaoFS.predict([padronizado[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = regressaoFS.predict([padronizado[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = regressaoFS.predict([padronizado[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = regressaoFS.predict([padronizado[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = regressaoFS.predict([padronizado[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = regressaoFS.predict([padronizado[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = regressaoFS.predict([padronizado[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = regressaoFS.predict([padronizado[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = regressaoFS.predict([padronizado[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = regressaoFS.predict([padronizado[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = regressaoFS.predict([padronizado[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = regressaoFS.predict([padronizado[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = regressaoFS.predict([padronizado[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = regressaoFS.predict([padronizado[0]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = regressaoFS.predict([padronizado[783]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 3.46509412 12.54361214]]
Este pokémon provavelmente é do tipo: electric/normal

Entrando com dados do tipo dark
[[ 6.17262973 11.35603459]]
Este pokémon provavelmente é do tipo: fire/ice

Entrando com dados do tipo dragon
[[ 5.72904026 12.89366803]]
Este pokémon provavelmente é do tipo: fighting/normal

Entrando com dados do tipo electric
[[ 6.44557585 14.5912642 ]]
Este pokémon provavelmente é do tipo: fire/psychic

Entrando com dados do tipo fairy
[[ 8.75597933 11.54578721]]
Este pokémon provavelmente é do tipo: ghost/ice

Entrando com dados do tipo fighting
[[ 6.60841033 13.38177025]]
Este pokémon provavelmente é do tipo: fire/poison

Entrando com dados do tipo fire
[[ 5.80673299 14.1929397 ]]
Este pokémon provavelmente é do tipo: fighting/psychic

Entrando com dados do tipo flying
[[10.4968991  10.18659985]]
Este pokémon provavelmente é do tipo: ground/ground

Entrando com dados do tipo ghost
[[ 9.82168236 11.11650847]]
Este pokémon provavelmente é

KNN COM FEATURE SCALING

In [50]:
knnFS = KNeighborsRegressor(n_neighbors=3)
knnFS.fit(entrada_treinoFS, saida_treinoFS)

KNeighborsRegressor(n_neighbors=3)

In [51]:
# mostra a precisão da regressão linear
print(f"Precisão: {knnFS.score(entrada_treinoFS, saida_treinoFS) * 100:.2f}%")

Precisão: 82.10%


In [52]:
predicao = knnFS.predict(entrada_testeFS)
print(f'MSE = {mean_squared_error(y_true=saida_testeFS, y_pred=predicao)}')  # raiz disso mostra uma média de desvio dos resultados (+-x)
print(f'R2 = {r2_score(y_true=saida_testeFS, y_pred=predicao)}')  # raiz disso mostra a porcentagem de acertos
print(f'MAE = {mean_absolute_error(y_true=saida_testeFS, y_pred=predicao)}')

MSE = 13.98412698412699
R2 = 0.565262954045364
MAE = 1.7474120082815736


In [53]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = knnFS.predict([padronizado[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = knnFS.predict([padronizado[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = knnFS.predict([padronizado[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = knnFS.predict([padronizado[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = knnFS.predict([padronizado[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = knnFS.predict([padronizado[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = knnFS.predict([padronizado[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = knnFS.predict([padronizado[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = knnFS.predict([padronizado[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = knnFS.predict([padronizado[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = knnFS.predict([padronizado[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = knnFS.predict([padronizado[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = knnFS.predict([padronizado[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = knnFS.predict([padronizado[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = knnFS.predict([padronizado[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = knnFS.predict([padronizado[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = knnFS.predict([padronizado[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = knnFS.predict([padronizado[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = knnFS.predict([padronizado[0]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = knnFS.predict([padronizado[783]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 0. 18.]]
Este pokémon provavelmente é do tipo: bug/pure

Entrando com dados do tipo dark
[[ 1. 18.]]
Este pokémon provavelmente é do tipo: dark/pure

Entrando com dados do tipo dragon
[[ 2. 18.]]
Este pokémon provavelmente é do tipo: dragon/pure

Entrando com dados do tipo electric
[[ 3. 18.]]
Este pokémon provavelmente é do tipo: electric/pure

Entrando com dados do tipo fairy
[[ 4. 18.]]
Este pokémon provavelmente é do tipo: fairy/pure

Entrando com dados do tipo fighting
[[ 5. 18.]]
Este pokémon provavelmente é do tipo: fighting/pure

Entrando com dados do tipo fire
[[ 6. 18.]]
Este pokémon provavelmente é do tipo: fire/pure

Entrando com dados do tipo flying
[[10.33333333 10.66666667]]
Este pokémon provavelmente é do tipo: ground/ground

Entrando com dados do tipo ghost
[[ 8. 18.]]
Este pokémon provavelmente é do tipo: ghost/pure

Entrando com dados do tipo grass
[[ 9. 18.]]
Este pokémon provavelmente é do tipo: grass/pure

Entrando com dados do ti

RANDOM FOREST COM FEATURE SCALING

In [54]:
rfFS = RandomForestRegressor()
rfFS.fit(entrada_treinoFS, saida_treinoFS)

RandomForestRegressor()

In [55]:
# mostra a precisão da regressão linear
print(f"Precisão: {rfFS.score(entrada_treinoFS, saida_treinoFS) * 100:.2f}%")

Precisão: 91.09%


In [56]:
predicao = rfFS.predict(entrada_testeFS)
print(f'MSE = {mean_squared_error(y_true=saida_testeFS, y_pred=predicao)}')  # raiz disso mostra uma média de desvio dos resultados (+-x)
print(f'R2 = {r2_score(y_true=saida_testeFS, y_pred=predicao)}')  # raiz disso mostra a porcentagem de acertos
print(f'MAE = {mean_absolute_error(y_true=saida_testeFS, y_pred=predicao)}')

MSE = 6.0420873629395535
R2 = 0.8106204883685593
MAE = 1.0957858294646898


In [57]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = rfFS.predict([padronizado[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = rfFS.predict([padronizado[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = rfFS.predict([padronizado[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = rfFS.predict([padronizado[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = rfFS.predict([padronizado[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = rfFS.predict([padronizado[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = rfFS.predict([padronizado[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = rfFS.predict([padronizado[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = rfFS.predict([padronizado[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = rfFS.predict([padronizado[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = rfFS.predict([padronizado[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = rfFS.predict([padronizado[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = rfFS.predict([padronizado[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = rfFS.predict([padronizado[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = rfFS.predict([padronizado[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = rfFS.predict([padronizado[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = rfFS.predict([padronizado[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = rfFS.predict([padronizado[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = rfFS.predict([padronizado[0]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = rfFS.predict([padronizado[783]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 0. 18.]]
Este pokémon provavelmente é do tipo: bug/pure

Entrando com dados do tipo dark
[[ 1. 18.]]
Este pokémon provavelmente é do tipo: dark/pure

Entrando com dados do tipo dragon
[[ 2. 18.]]
Este pokémon provavelmente é do tipo: dragon/pure

Entrando com dados do tipo electric
[[ 3.         17.23846923]]
Este pokémon provavelmente é do tipo: electric/water

Entrando com dados do tipo fairy
[[ 4. 18.]]
Este pokémon provavelmente é do tipo: fairy/pure

Entrando com dados do tipo fighting
[[ 5. 18.]]
Este pokémon provavelmente é do tipo: fighting/pure

Entrando com dados do tipo fire
[[ 6.        17.1354882]]
Este pokémon provavelmente é do tipo: fire/water

Entrando com dados do tipo flying
[[ 9.09 13.82]]
Este pokémon provavelmente é do tipo: grass/poison

Entrando com dados do tipo ghost
[[ 8. 18.]]
Este pokémon provavelmente é do tipo: ghost/pure

Entrando com dados do tipo grass
[[ 9. 18.]]
Este pokémon provavelmente é do tipo: grass/pure

Entra

DECISION TREE COM FEATURE SCALING

In [58]:
dtrFS = DecisionTreeRegressor()
dtrFS.fit(entrada_treinoFS, saida_treinoFS)

DecisionTreeRegressor()

In [59]:
# mostra a precisão da regressão linear
print(f"Precisão: {dtrFS.score(entrada_treinoFS, saida_treinoFS) * 100:.2f}%")

Precisão: 92.06%


In [60]:
predicao = dtrFS.predict(entrada_testeFS)
print(f'MSE = {mean_squared_error(y_true=saida_testeFS, y_pred=predicao)}')  # raiz disso mostra uma média de desvio dos resultados (+-x)
print(f'R2 = {r2_score(y_true=saida_testeFS, y_pred=predicao)}')  # raiz disso mostra a porcentagem de acertos
print(f'MAE = {mean_absolute_error(y_true=saida_testeFS, y_pred=predicao)}')

MSE = 7.005686400309925
R2 = 0.7787241041273878
MAE = 0.9526059945316734


In [61]:
# vantagens e desvantagens para o tipo inseto
print(f'Entrando com dados do tipo {tipagem[0]}')
predicao = dtrFS.predict([padronizado[9]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo sombrio
print(f'Entrando com dados do tipo {tipagem[1]}')
predicao = dtrFS.predict([padronizado[196]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão
print(f'Entrando com dados do tipo {tipagem[2]}')
predicao = dtrFS.predict([padronizado[146]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo elétrico
print(f'Entrando com dados do tipo {tipagem[3]}')
predicao = dtrFS.predict([padronizado[24]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fada
print(f'Entrando com dados do tipo {tipagem[4]}')
predicao = dtrFS.predict([padronizado[34]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo luta
print(f'Entrando com dados do tipo {tipagem[5]}')
predicao = dtrFS.predict([padronizado[55]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fogo
print(f'Entrando com dados do tipo {tipagem[6]}')
predicao = dtrFS.predict([padronizado[3]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo voador
print(f'Entrando com dados do tipo {tipagem[7]}')
predicao = dtrFS.predict([padronizado[640]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo fantasma
print(f'Entrando com dados do tipo {tipagem[8]}')
predicao = dtrFS.predict([padronizado[199]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama
print(f'Entrando com dados do tipo {tipagem[9]}')
predicao = dtrFS.predict([padronizado[113]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo solo
print(f'Entrando com dados do tipo {tipagem[10]}')
predicao = dtrFS.predict([padronizado[26]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo gelo
print(f'Entrando com dados do tipo {tipagem[11]}')
predicao = dtrFS.predict([padronizado[360]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo normal
print(f'Entrando com dados do tipo {tipagem[12]}')
predicao = dtrFS.predict([padronizado[18]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo venenoso
print(f'Entrando com dados do tipo {tipagem[13]}')
predicao = dtrFS.predict([padronizado[22]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo psiquico
print(f'Entrando com dados do tipo {tipagem[14]}')
predicao = dtrFS.predict([padronizado[62]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo pedra
print(f'Entrando com dados do tipo {tipagem[15]}')
predicao = dtrFS.predict([padronizado[184]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo aço
print(f'Entrando com dados do tipo {tipagem[16]}')
predicao = dtrFS.predict([padronizado[378]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo água
print(f'Entrando com dados do tipo {tipagem[17]}')
predicao = dtrFS.predict([padronizado[6]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int((predicao[0][0]))]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo grama/venenoso
print('Entrando com dados do tipo grama/venenoso')
predicao = dtrFS.predict([padronizado[0]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

# vantagens e desvantagens para o tipo dragão/lutador
print('Entrando com dados do tipo dragão/lutador')
predicao = dtrFS.predict([padronizado[783]])
print(predicao)
print(f'Este pokémon provavelmente é do tipo: \033[1m{tipagem[int(predicao[0][0])]}/{tipagem[int(predicao[0][1])]}\033[m\n')

Entrando com dados do tipo bug
[[ 0. 18.]]
Este pokémon provavelmente é do tipo: bug/pure

Entrando com dados do tipo dark
[[ 1. 18.]]
Este pokémon provavelmente é do tipo: dark/pure

Entrando com dados do tipo dragon
[[ 2. 18.]]
Este pokémon provavelmente é do tipo: dragon/pure

Entrando com dados do tipo electric
[[ 3.         17.21052632]]
Este pokémon provavelmente é do tipo: electric/water

Entrando com dados do tipo fairy
[[ 4. 18.]]
Este pokémon provavelmente é do tipo: fairy/pure

Entrando com dados do tipo fighting
[[ 5. 18.]]
Este pokémon provavelmente é do tipo: fighting/pure

Entrando com dados do tipo fire
[[ 6.         17.17391304]]
Este pokémon provavelmente é do tipo: fire/water

Entrando com dados do tipo flying
[[ 7. 18.]]
Este pokémon provavelmente é do tipo: flying/pure

Entrando com dados do tipo ghost
[[ 8. 18.]]
Este pokémon provavelmente é do tipo: ghost/pure

Entrando com dados do tipo grass
[[ 9. 18.]]
Este pokémon provavelmente é do tipo: grass/pure

Entrando